# Test modules using Polytropes #
Test modules with M, rho, P from polytrope solution and using $P = \rho \frac{R}{\mu}T$ as the equation of state

In [ ]:
%pylab ipympl
ifig = 0

import polytrope
import opacity.opal as opal
import nuclear
import scipy.interpolate

# Polytrope and Equation of state #

In [ ]:
n = 3
rho_c = 150 # [g/cm^3]
P_c = 2.5e17 # [dyn/cm^2]
X, Z = 0.7, 0.02 # mass fraction
data = polytrope.solve_polytrope(rho_c, P_c, n, phi_range=(0, 1000), atol=1e-10, rtol=1e-10)#, phi_range=(0, 100)
m, r, rho, P = polytrope.polytrope_to_cgs(data, rho_c, P_c, n)

R = 8.314e7 # in [erg/(g K)]
mu = 4/(5*X + 3 - Z) # unitless

T = P*mu/(rho*R) # in [K]
T_c = T.max()
print(f"Core tempurature: {T_c} in [K]")

rho = numpy.abs(rho)
P = numpy.abs(P)
T = numpy.abs(T)

## opacity ##

In [ ]:
opal_data = opal.read_file("opacity/tables/GN93hz")
opacity = opal.linear_opacity(opal_data)
k_R = opacity(rho, T, X, Z) # in (cm^2/g)
mfp = 1/(rho*k_R)

## nuclear network ##

In [ ]:
reactions = nuclear.network.read_network_file("nuclear/networks/pp_branch_I") # + nuclear.network.read_network_file("nuclear/networks/cno_branch_I")
ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
nuclear.network.draw_network(reactions, show=False)
pyplot.show(ifig)

particles, get_dYdt_epsilon = nuclear.network.build_network(reactions, "./nuclear/reactions")
print(f"particles: {particles}")

Y = 1 - X - Z

X_0 = [Z/(len(particles) - 2)]*len(particles)
X_0[0], X_0[3] = X, Y
#X_0[1], X_0[2] = 1e-17, 1e-5

Y_0 = [x/AZ[0] for x, AZ in zip(X_0, particles)]
epsilon = numpy.zeros(shape=T.shape)
for i in range(len(T)):
    dYdt, epsilon_fun = get_dYdt_epsilon(rho[i], T[i])
    epsilon[i] = epsilon_fun(0, Y_0)

L = numpy.zeros(shape=T.shape)
L[1:] = numpy.cumsum(epsilon[1:]*numpy.diff(m))

power_density = rho*epsilon


In [ ]:
ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(m/m.max(), r/r.max(), label="radius $y = R$")
pyplot.plot(m/m.max(), rho/rho.max(), label="density $y = \\rho$")
pyplot.plot(m/m.max(), P/P.max(), label="pressure $y = P$")
pyplot.plot(m/m.max(), T/T.max(), label="tempurature $y = T$")
pyplot.plot(m/m.max(), L/L.max(), label="luminosity $y = L$")
pyplot.plot(m/m.max(), k_R/k_R.max(), label="mean opacity $y = k_R$")
pyplot.plot(m/m.max(), power_density/power_density.max(), label="power density $y = \\varepsilon \\rho$")
pyplot.xlabel("mass coordinate")
pyplot.ylabel("parameter $y$ fraction of maximum")
pyplot.legend()
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r, m/m.max(), label="mass $y = M$")
pyplot.plot(r, rho/rho.max(), label="density $y = \\rho$")
pyplot.plot(r, P/P.max(), label="pressure $y = P$")
pyplot.plot(r, T/T.max(), label="tempurature $y = T$")
pyplot.plot(r, L/L.max(), label="luminosity $y = L$")
pyplot.plot(r, k_R/k_R.max(), label="mean opacity $y = k_R$")
pyplot.plot(r, power_density/power_density.max(), label="power density $y = \\varepsilon \\rho$")
pyplot.xlabel("radius [cm]")
pyplot.ylabel("parameter $y$ fraction of maximum")
pyplot.legend()
pyplot.show(ifig)

In [ ]:
ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r, rho)
pyplot.xlabel("radius [cm]")
pyplot.ylabel("rho [g/cm^3]")
pyplot.title("density")
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r, T)
pyplot.xlabel("radius [cm]")
pyplot.ylabel("temperature [K]")
pyplot.title("Temperature")
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.semilogy(r, mfp)
pyplot.xlabel("radius [cm]")
pyplot.ylabel("mean free path [cm]")
pyplot.title("Opacity: Mean free path of light")
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r, power_density)
pyplot.xlabel("radius [cm]")
pyplot.ylabel("power density [erg/(s cm^3)]")
pyplot.title("Nuclear power density")
pyplot.show(ifig)

## Nuclear Network Array ##

In [ ]:
import scipy.integrate

Y_0 = [x/AZ[0] for x, AZ in zip(X_0, particles)]
Y_0 = numpy.array(Y_0)[None, :] + 0*T[:, None]

t_range = (0, 1*60*60*24*365.25)

Y = Y_0
for i in range(len(T)):
    T_i = T[i]
    rho_i = rho[i]
    Y_i = Y[i]
    
    dYdt, epsilon = get_dYdt_epsilon(rho_i, T_i)
    solve = scipy.integrate.solve_ivp(dYdt, t_range, Y_i, method="Radau")#, rtol=1e-9, atol=1e-9)
    print(solve.message)

    _t = solve["t"]/(60*60*24*365.25) # convert time to years from seconds
    _Y = solve["y"]
    _X = [y*AZ[0] for y, AZ in zip(_Y, particles)]

    if i % (len(T)//5) == 0:
        ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
        for x,AZ in zip(_X, particles):
            pyplot.loglog(_t, x, label=nuclear.isotopes.AZN_to_str((*AZ, 1)))
        pyplot.xlabel("t [yr]")
        pyplot.ylabel("mass fraction")
        pyplot.title(f"PP-Chain at fixed $T$ {T_i:.3g}, $\\rho$ {rho_i:.3g}, {i:d}")
        pyplot.ylim(1e-10, 1.5)
        pyplot.legend()
        pyplot.show(ifig)
    Y[i] = _Y[:, -1]

In [ ]:
X = numpy.array([y*AZ[0] for y, AZ in zip(Y.T, particles)])
print(X.shape, T.shape)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
for x,AZ in zip(X, particles):
    pyplot.semilogy(m/m.max(), x, label=nuclear.isotopes.AZN_to_str((*AZ, 1)))
pyplot.xlabel("mass coordinate")
pyplot.ylabel("mass fraction")
pyplot.title(f"PP-Chain")
pyplot.ylim(1e-10, 1.5)
pyplot.legend()
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
for x,AZ in zip(X, particles):
    pyplot.semilogy(r, x, label=nuclear.isotopes.AZN_to_str((*AZ, 1)))
pyplot.xlabel("radius in [cm]")
pyplot.ylabel("mass fraction")
pyplot.title(f"PP-Chain")
pyplot.ylim(1e-10, 1.5)
pyplot.legend()
pyplot.show(ifig)

In [ ]:
Y_prime = copy(Y)
#rint(Y_prime)

In [ ]:
t_range = (0, 5e5*60*60*24*365.25)

Y = copy(Y_prime)
for i in range(len(T)):
    T_i = T[i]
    rho_i = rho[i]
    Y_i = Y[i]
    
    dYdt, epsilon = get_dYdt_epsilon(rho_i, T_i)
    solve = scipy.integrate.solve_ivp(dYdt, t_range, Y_i, method="Radau", atol=1e-8, rtol=1e-8)
    print(solve.message)

    _t = solve["t"]/(60*60*24*365.25) # convert time to years from seconds
    _Y = solve["y"]
    _X = [y*AZ[0] for y, AZ in zip(_Y, particles)]

    if i % (len(T)//5) == 0:
        ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
        for x,AZ in zip(_X, particles):
            pyplot.loglog(_t, x, label=nuclear.isotopes.AZN_to_str((*AZ, 1)))
        pyplot.xlabel("t [yr]")
        pyplot.ylabel("mass fraction")
        pyplot.title(f"PP-Chain at fixed $T$ {T_i:.3g}, $\\rho$ {rho_i:.3g}, {i:d}")
        pyplot.ylim(1e-10, 1.5)
        pyplot.legend()
        pyplot.show(ifig)
    Y[i] = _Y[:, -1]

In [ ]:
Y = numpy.abs(Y)
X = numpy.array([y*AZ[0] for y, AZ in zip(Y.T, particles)])
print(X.shape, m.shape)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
for x,AZ in zip(X, particles):
    pyplot.semilogy(m/m.max(), x, label=nuclear.isotopes.AZN_to_str((*AZ, 1)))
pyplot.xlabel("mass coordinate")
pyplot.ylabel("mass fraction")
pyplot.title(f"PP-Chain")
#pyplot.ylim(1e-10, 1.5)
pyplot.legend()
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
for x,AZ in zip(X, particles):
    pyplot.semilogy(r, x, label=nuclear.isotopes.AZN_to_str((*AZ, 1)))
pyplot.xlabel("radius in [cm]")
pyplot.ylabel("mass fraction")
pyplot.title(f"PP-Chain")
#pyplot.ylim(1e-10, 1.5)
pyplot.legend()
pyplot.show(ifig)

logX = [scipy.interpolate.interp1d(m, numpy.log10(X_i), fill_value="extrapolate") for X_i in X]

In [ ]:
k_R = opacity(rho, T, X, Z) # in (cm^2/g)
mfp = 1/(rho*k_R)

epsilon = numpy.zeros(shape=T.shape)
for i in range(len(T)):
    dYdt, epsilon_fun = get_dYdt_epsilon(rho[i], T[i])
    epsilon[i] = epsilon_fun(0, Y[i])

L = numpy.zeros(shape=T.shape)
L[1:] = numpy.cumsum(epsilon[1:]*numpy.diff(m))

power_density = rho*epsilon

In [ ]:
ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(m/m.max(), r/r.max(), label="radius $y = R$")
pyplot.plot(m/m.max(), rho/rho.max(), label="density $y = \\rho$")
pyplot.plot(m/m.max(), P/P.max(), label="pressure $y = P$")
pyplot.plot(m/m.max(), T/T.max(), label="tempurature $y = T$")
pyplot.plot(m/m.max(), L/L.max(), label="luminosity $y = L$")
pyplot.plot(m/m.max(), k_R/k_R.max(), label="mean opacity $y = k_R$")
pyplot.plot(m/m.max(), power_density/power_density.max(), label="power density $y = \\varepsilon \\rho$")
pyplot.xlabel("mass coordinate")
pyplot.ylabel("parameter $y$ fraction of maximum")
pyplot.legend()
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r, m/m.max(), label="mass $y = M$")
pyplot.plot(r, rho/rho.max(), label="density $y = \\rho$")
pyplot.plot(r, P/P.max(), label="pressure $y = P$")
pyplot.plot(r, T/T.max(), label="tempurature $y = T$")
pyplot.plot(r, L/L.max(), label="luminosity $y = L$")
pyplot.plot(r, k_R/k_R.max(), label="mean opacity $y = k_R$")
pyplot.plot(r, power_density/power_density.max(), label="power density $y = \\varepsilon \\rho$")
pyplot.xlabel("radius [cm]")
pyplot.ylabel("parameter $y$ fraction of maximum")
pyplot.legend()
pyplot.show(ifig)

In [ ]:
ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r, power_density)
pyplot.xlabel("radius [cm]")
pyplot.ylabel("power density [erg/(s cm^3)]")
pyplot.title("Nuclear power density")
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.semilogy(r, mfp)
pyplot.xlabel("radius [cm]")
pyplot.ylabel("mean free path [cm]")
pyplot.title("Opacity: Mean free path of light")
pyplot.show(ifig)

In [ ]:
sigma_sb = 5.67037442e-5 # in [erg/(s cm^2 K^4)]

logX[1] = lambda m:-20

def dPrLTdM(M, PrLT):
    P, r, L, T = numpy.abs(PrLT)
    #if any(numpy.isnan(PrLT)):
    #    print(P, r, L, T)
    
    # mean molecular weight
    mu = 4/(5*0.7 + 3 - 0.02) # unetless
#    mu = 4/(3*0.7 + 1 - 0.02) # unetless
    rho = mu*P/(R*T) # in [g/cm^3]
#    rho = 150*(P/2.5e17)**(3/4)
    
    
    _, epsilon_fun = get_dYdt_epsilon(rho, T)
    epsilon = epsilon_fun(0, [10**logx(M) for logx in logX])
    k_R = opacity([rho], [T], X, Z)[0]*0.01

    dPdM = -polytrope.G*M/(4*numpy.pi*r**4)
    drdM = 1/(4*numpy.pi*r**2*rho)
    dLdM = epsilon*0.5 # 0.5 for neutreno losses
    dTdM = -0.0001*3*k_R*L/(256*numpy.pi**2*r**4*sigma_sb*T**3)
    return [dPdM, drdM, dLdM, dTdM]

def zP(M, PrLT):
    P, r, L, T = PrLT
    return P
zP.terminal = True

def zT(M, PrLT):
    P, r, L, T = PrLT
    return T
zT.terminal = True

def BC(M, PrLT):
    P_c, r_c, L_c, T_c = PrLT[:, 0]
    P_s, r_s, L_s, T_s = PrLT[:, -1]
    M_s = M[-1]
    
#    rho_s = mu*P_s/(R*T_s)
    rho_s = 150*(P_s/2.5e17)**(3/4)
    k_R = opacity([rho_s], [T_s], X, Z)[0]

    M_R = m[-1]
    P_R = (2/3)*polytrope.G*M_s/(r_s**2*k_R)
    L_R = 4*numpy.pi*r_s**2*sigma_sb*T_s**4
    
    return [M_s/M_R, (P_s - P_R)/P_R, (L_s - L_R)/L_R]

PrLT_0 = [P[0], 1e-5, 0.0, T[0]]
events = [zP, zT]
print(PrLT_0)
solve = scipy.integrate.solve_ivp(dPrLTdM, (m[0], 2*m[-1]), PrLT_0, events=events, method="Radau", atol=1e-6, rtol=1e-6)
print(solve.message)
PrLT = solve["y"]
M = solve["t"]
print("BC " + ", ".join([f"{v:.3e}" for v in BC(M, PrLT)]))
print(numpy.sum(numpy.array(BC(M, PrLT))**2))

In [ ]:
r, n = 1.5, 5
values = numpy.zeros(shape=(n, n))
for i, a_P in enumerate(10**numpy.linspace(-4*r, 4*r, n)):
    for j, a_T in enumerate(10**numpy.linspace(-r, r, n)):
        PrLT_0 = [a_P*P[0], 1e-5, 0.0, a_T*T[0]]
        try:
            solve = scipy.integrate.solve_ivp(dPrLTdM, (m[0], m[-1]), PrLT_0, method="Radau")
            print(", ".join([f"{v:11.3e}" for v in BC(M, PrLT)]), i, j, solve.message)
            PrLT = solve["y"]
            M = solve["t"]
            values[i, j] = numpy.sum(numpy.array(BC(M, PrLT))**2)
        except:
            values[i, j] = float("Nan")

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.imshow(numpy.log10(values))
pyplot.show(ifig)

In [ ]:
M1 = None
M1 = solve["t"]
P1, r1, L1, T1 = solve["y"]

rho1 = mu*P1/(R*T1)
k_R1 = opacity(rho1, T1, X, Z)
mpl1 = 1/(rho1*k_R1)

P_t = (2/3)*polytrope.G*M1[-1]/(r1[-1]**2*k_R1[-1])
T_t = (L1[-1]/(4*numpy.pi*r1[-1]**2*sigma_sb))**(1/4)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(M1/M1.max(), r1/r1.max(), label="radius $y = R$")
#pyplot.plot(M1/M1.max(), rho1/rho1.max(), marker="s", label="density $y = \\rho$")
pyplot.plot(M1/M1.max(), P1/P1.max(), label="pressure $y = P$")
pyplot.plot(M1/M1.max(), T1/T1.max(), marker="*", label="tempurature $y = T$")
pyplot.plot(M1/M1.max(), L1/L1.max(), label="luminosity $y = L$")
#pyplot.plot(M1/M1.max(), k_R1/k_R1.max(), label="opacity")
#pyplot.plot(M1/M1.max(), mpl1/mpl1.max(), label="mpl")

pyplot.hlines(T_t/T1.max(), 0, 1, "r", "--", label="T target")
pyplot.hlines(P_t/P1.max(), 0, 1, "g", ":", label="T target")
#pyplot.plot(m/m.max(), k_R/k_R.max(), label="mean opacity $y = k_R$")
#pyplot.plot(m/m.max(), power_density/power_density.max(), label="power density $y = \\varepsilon \\rho$")
pyplot.xlabel("mass coordinate")
pyplot.ylabel("parameter $y$ fraction of maximum")
pyplot.legend()
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r1, M1/M1.max(), label="mass $y = R$")
#pyplot.plot(r1, rho1/rho1.max(), marker="s", label="density $y = \\rho$")
pyplot.plot(r1, P1/P1.max(), label="pressure $y = P$")
pyplot.plot(r1, T1/T1.max(), marker="*", label="tempurature $y = T$")
pyplot.plot(r1, L1/L1.max(), label="luminosity $y = L$")
#pyplot.plot(r1, k_R1/k_R1.max(), label="opacity")
#pyplot.plot(r1, mpl1/mpl1.max(), label="mpl")
#pyplot.plot(m/m.max(), k_R/k_R.max(), label="mean opacity $y = k_R$")
#pyplot.plot(m/m.max(), power_density/power_density.max(), label="power density $y = \\varepsilon \\rho$")
pyplot.xlabel("radius in cm")
pyplot.ylabel("parameter $y$ fraction of maximum")
pyplot.legend()
pyplot.show(ifig)